In [4]:
# for gioia's python build
import sys
sys.path.append('/Users/maggiori/anaconda/envs/py35/lib/python3.5/site-packages')

In [5]:
import numpy as np

In [6]:
# to import modules locally without having installed the entire package
# http://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import os, sys, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

In [7]:
from tsdb import PersistentDB
from tsdb import BitMapIndex
from timeseries import TimeSeries

Generating LALR tables


In [8]:
# synthetic data
t = np.array([1, 1.5, 2, 2.5, 10, 11, 12])
v1 = np.array([10, 12, -11, 1.5, 10, 13, 17])
v2 = np.array([8, 12, -11, 1.5, 10, 13, 17])
a1 = TimeSeries(t, v1)
a2 = TimeSeries(t, v2)

In [9]:
identity = lambda x: x

# index: 1 is binary tree index, 2 is bitmap index
schema = {
  'pk':         {'type': 'str', 'convert': identity,   'index': None, 'values': None},
  'ts':         {'type': 'int', 'convert': identity,   'index': None, 'values': None},
  'order':      {'type': 'int', 'convert': int,        'index': 1, 'values': None},
  'blarg':      {'type': 'int', 'convert': int,        'index': 1, 'values': None},
  'useless':    {'type': 'int', 'convert': identity,   'index': 1, 'values': None},
  'mean':       {'type': 'float', 'convert': float,    'index': 1, 'values': None},
  'std':        {'type': 'float', 'convert': float,    'index': 1, 'values': None},
  'vp':         {'type': 'bool', 'convert': bool,      'index': 2, 'values': [True, False]},
  'deleted':    {'type': 'bool', 'convert': bool,      'index': 2, 'values': [True, False]}
}

In [10]:
# # Delete any default db present (otherwise the db creation will load
# # the previous one...)
# filelist = ["db_files/" + f for f in os.listdir("db_files/")
#             if f[:7] == 'default']
# for f in filelist:
#     os.remove(f)

In [11]:
# create persistent db
ddb = PersistentDB(schema, 'pk', len(t))

After init pks:  {'pk4': (788, 175), 'pk6': (564, 125), 'pk2': (116, 25), 'pk5': (900, 200), 'pk3': (1012, 225), 'pk7': (676, 150), 'pk1': (228, 50)}


In [12]:
ddb.indexes['vp'].values()

TypeError: 'list' object is not callable

In [9]:
ddb.insert_ts('pk1', a1)

Before insert, state of pks:  {}
After insert, state of pks:  {'pk1': (4, 0)}


In [10]:
ddb.upsert_meta('pk1', {'order': 1, 'blarg': 2})

In [11]:
ddb.insert_ts('pk2', a2)

Before insert, state of pks:  {'pk1': (4, 0)}
After insert, state of pks:  {'pk1': (4, 0), 'pk2': (116, 25)}


In [12]:
ddb.upsert_meta('pk2', {'order': 2, 'blarg': 2})

In [13]:
ddb.indexes

{'blarg': <tsdb.indexes.BinTreeIndex at 0x105d7c2b0>,
 'deleted': <tsdb.indexes.BitMapIndex at 0x105d7cb38>,
 'mean': <tsdb.indexes.BinTreeIndex at 0x105d7c940>,
 'order': <tsdb.indexes.BinTreeIndex at 0x105d7c470>,
 'std': <tsdb.indexes.BinTreeIndex at 0x105d7c0b8>,
 'useless': <tsdb.indexes.BinTreeIndex at 0x105d7ccc0>,
 'vp': <tsdb.indexes.BitMapIndex at 0x105d7cd30>}

In [17]:
# print metadata
for field, index in ddb.indexes.items():
    if isinstance(index, BitMapIndex):
        print(field)
        for k in index.keys():
            print(k, index[k])
        print('-----')

vp
False {'pk1', 'pk2'}
True set()
-----
deleted
False {'pk1', 'pk2'}
True set()
-----


In [18]:
ddb.pks.keys()

dict_keys(['pk1', 'pk2'])

In [19]:
ddb.pks.keys()

dict_keys(['pk1', 'pk2'])

In [21]:
# print metadata
for field, index in ddb.indexes.items():
    if isinstance(index, BitMapIndex):
        print(field)
        for k in index.keys():
            print(k, index[k])
        print('-----')

vp
False {'pk1', 'pk2'}
True set()
-----
deleted
False {'pk1', 'pk2'}
True set()
-----


In [22]:
ddb.select({'pk': 'pk1'}, [], None)

(['pk1'],
 [{'blarg': 2,
   'mean': 0.0,
   'order': 1,
   'pk': 'pk1',
   'std': 0.0,
   'useless': 0,
   'vp': False}])

In [23]:
ddb.select({'pk': 'pk2'}, [], None)

(['pk2'],
 [{'blarg': 2,
   'mean': 0.0,
   'order': 2,
   'pk': 'pk2',
   'std': 0.0,
   'useless': 0,
   'vp': False}])

In [24]:
ddb.delete_ts('pk1')

In [25]:
ddb.insert_ts('pk1', a1)

Before insert, state of pks:  {'pk2': (116, 25)}
After insert, state of pks:  {'pk1': (228, 50), 'pk2': (116, 25)}


In [26]:
# print metadata
for field, index in ddb.indexes.items():
    if isinstance(index, BitMapIndex):
        print(field)
        for k in index.keys():
            print(k, index[k])
        print('-----')

vp
False {'pk1', 'pk2'}
True set()
-----
deleted
False {'pk1', 'pk2'}
True set()
-----


In [44]:
ddb.insert_ts('pk3', a1)
ddb.insert_ts('pk4', a1)
ddb.insert_ts('pk5', a1)
ddb.insert_ts('pk6', a1)
ddb.insert_ts('pk7', a1)

Before insert, state of pks:  {'pk6': (564, 125), 'pk4': (788, 175), 'pk5': (900, 200), 'pk7': (676, 150), 'pk1': (228, 50), 'pk2': (116, 25)}
After insert, state of pks:  {'pk6': (564, 125), 'pk4': (788, 175), 'pk3': (1012, 225), 'pk5': (900, 200), 'pk7': (676, 150), 'pk1': (228, 50), 'pk2': (116, 25)}


In [28]:
# print metadata
for field, index in ddb.indexes.items():
    if isinstance(index, BitMapIndex):
        print(field)
        for k in index.keys():
            print(k, index[k])
        print('-----')

vp
False {'pk6', 'pk4', 'pk7', 'pk1', 'pk2', 'pk5'}
True set()
-----
deleted
False {'pk6', 'pk4', 'pk7', 'pk1', 'pk2', 'pk5'}
True set()
-----


In [29]:
ddb.upsert_meta('pk4', {'vp': True})
for field, index in ddb.indexes.items():
    if field == 'vp':
        for k in index.keys():
            print(k, index[k])

False {'pk6', 'pk5', 'pk7', 'pk1', 'pk2'}
True {'pk4'}


In [30]:
ddb.upsert_meta('pk5', {'vp': True})
for field, index in ddb.indexes.items():
    if field == 'vp':
        for k in index.keys():
            print(k, index[k])

False {'pk6', 'pk7', 'pk1', 'pk2'}
True {'pk5', 'pk4'}


In [31]:
ddb.upsert_meta('pk5', {'vp': False})
for field, index in ddb.indexes.items():
    if field == 'vp':
        for k in index.keys():
            print(k, index[k])

False {'pk6', 'pk5', 'pk7', 'pk1', 'pk2'}
True {'pk4'}


In [32]:
ddb.upsert_meta('pk5', {'vp': True})
for field, index in ddb.indexes.items():
    if field == 'vp':
        for k in index.keys():
            print(k, index[k])

False {'pk6', 'pk7', 'pk1', 'pk2'}
True {'pk5', 'pk4'}


In [33]:
ddb.delete_ts('pk5')

In [34]:
for field, index in ddb.indexes.items():
    if field == 'vp':
        for k in index.keys():
            print(k, index[k])

False {'pk6', 'pk7', 'pk1', 'pk2'}
True {'pk4'}


In [35]:
ddb.delete_ts('pk4')

In [36]:
for field, index in ddb.indexes.items():
    if field == 'vp':
        for k in index.keys():
            print(k, index[k])

False {'pk6', 'pk7', 'pk1', 'pk2'}
True set()


In [37]:
ddb.insert_ts('pk4', a1)
ddb.insert_ts('pk5', a1)
ddb.upsert_meta('pk4', {'vp': True})
ddb.upsert_meta('pk5', {'vp': True})

Before insert, state of pks:  {'pk6': (564, 125), 'pk7': (676, 150), 'pk1': (228, 50), 'pk2': (116, 25)}
After insert, state of pks:  {'pk6': (564, 125), 'pk4': (788, 175), 'pk7': (676, 150), 'pk1': (228, 50), 'pk2': (116, 25)}
Before insert, state of pks:  {'pk6': (564, 125), 'pk4': (788, 175), 'pk7': (676, 150), 'pk1': (228, 50), 'pk2': (116, 25)}
After insert, state of pks:  {'pk6': (564, 125), 'pk4': (788, 175), 'pk5': (900, 200), 'pk7': (676, 150), 'pk1': (228, 50), 'pk2': (116, 25)}


In [45]:
ddb.select({'vp': True}, ['vp'], None)

(['pk5', 'pk4'], [{'vp': True}, {'vp': True}])

In [46]:
ddb.select({'vp': False}, ['vp'], None)

(['pk6', 'pk3', 'pk7', 'pk1', 'pk2'],
 [{'vp': False}, {'vp': False}, {'vp': False}, {'vp': False}, {'vp': False}])

In [47]:
ddb = None